# Проект для «Викишоп» с BERT


## Введение<a id="intro"></a>


Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

План работы: 
1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Вывод.

Описание данных:
 - *text*  - текст комментария, 
 - *toxic* - целевой признак.


## Оглавление:

1. [Введение](#intro)

2. [Подготовка](#preparing)
 * [2.1 Открытие и изучиние файлов](#open)
 * [2.2 Подготовка данных к обучению](#preparation)
 

3. [Обучение](#training)
 * [3.1 Логистическая регрессия без кросс-валидации](#3.1)
 * [3.2 Логистическая регрессия с кросс-валидацией](#3.2)
 * [3.3 Случайный лес](#3.3)
 * [3.4 Дерево решений](#3.4)
 
 
4. [Вывод](#conclusion)



## 1. Подготовка<a id="preparing"></a>
### 1.1 Открытие и изучиние файлов<a id="open"></a>

In [1]:
# Импорт библеотек

from IPython.display import display
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
from tqdm import notebook
from sklearn.tree import DecisionTreeClassifier

In [2]:
# Открытие данных
df = pd.read_csv('toxic_comments.csv')
display(df.head(5))

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
df.info()
df.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


0

В загруженном датасете 159571 записей. Пропусков нет, дубликатов нет.

### 1.2 Подготовка данных к обучению<a id="preparation"></a>
#### Разбиение датасета на тестовые и тренировочные выборки.

In [6]:
x_train, x_test, y_train, y_test = train_test_split(
    df.text, df.toxic, test_size=0.25, random_state=12345)
print('Проверка выборок:', x_train.shape, x_train.shape, x_test.shape, y_test.shape)

Проверка выборок: (119678,) (119678,) (39893,) (39893,)


#### Вычисление TF-IDF
Для тренировочной выборки

In [7]:
corpus = x_train.values.astype('U')
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit_transform(corpus)
tf_idf.shape

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sugan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(119678, 159902)

Для тестовой выборки

In [8]:
corpus = x_test.values.astype('U')
tf_idf_test = count_tf_idf.transform(corpus)
tf_idf_test.shape

(39893, 159902)

## 2. Обучение <a id='training'></a>
### 2.1 Логистическая регрессия без кросс-валидации<a id="3.1"></a>

In [9]:
model = LogisticRegression(random_state=12345, class_weight='balanced').fit(tf_idf, y_train)
predictions = model.predict(tf_idf_test)
f1 = f1_score(y_test, predictions)
print('f1 для логистической регрессии без кросс-валидации:', f1)

# Список показателей f1 моделей
f1_list = []
f1_list.append(f1.round(2))

D:\Program Files\anaconda3\envs\praktikum\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


f1 для логистической регрессии без кросс-валидации: 0.7596426236652866


### 2.2 Логистическая регрессия с кросс-валидацией<a id="3.2"></a>

In [10]:
model =  LogisticRegression(class_weight='balanced')
f1 = cross_val_score(model, tf_idf, y_train, cv=5, scoring=make_scorer(f1_score))
f1 = f1.mean()
print('f1 для логистической регрессии c кросс-валидацией:', f1)


# запись f1
f1_list.append(f1.round(2))

D:\Program Files\anaconda3\envs\praktikum\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


f1 для логистической регрессии c кросс-валидацией: 0.7492306764906909


### 2.3 Случайный лес<a id="3.3"></a>

In [11]:
best_model = None
best_f1 = 0
for depth in notebook.tqdm(range(50, 150, 10)):
    model = RandomForestClassifier(max_depth=depth, random_state=12345, n_estimators=1)
    f1 = cross_val_score(model, tf_idf, y_train, cv=5, scoring=make_scorer(f1_score))
    f1 = f1.mean()
    if best_f1 < f1:
        best_model = model
        best_f1 = f1
        best_depth = depth
        
            
print('Налилучший показатель F1 = {} для случайного леса достигается при глубине равной {}:'.format(best_f1,best_depth) )

# запись f1
f1_list.append(f1.round(2))

  0%|          | 0/4 [00:00<?, ?it/s]

Налилучший показатель F1 = 0.4243484278269321 для случайного леса достигается при глубине равной 125:


### 2.4 Дерево решений<a id="3.4"></a>

In [12]:
best_depth = 0
best_model = None
best_f1 = 0

for depth in notebook.tqdm(range(50, 150, 10)):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth).fit(tf_idf, y_train)
    predictions = model.predict(tf_idf_test)
    f1 = f1_score(y_test, predictions)
    if best_f1 < f1:
        best_f1 = f1
        best_depth = depth
        best_model = model

print('Налилучший показатель F1 = {} для дерева решений достигается при глубине равной {}:'.format(best_f1,best_depth) )

# запись f1
f1_list.append(f1.round(2))

  0%|          | 0/10 [00:00<?, ?it/s]

Налилучший показатель F1 = 0.7273696840421277 для дерева решений достигается при глубине равной 120:


In [13]:
index_list = ['Логистическая регрессия без кросс-валидации', 'Логистическая регрессия с кросс-валидацией', 'Дерево решений', "Случайный лес"]
d = {'f1': f1_list}
result = pd.DataFrame(data=d, index = index_list)

,f1
Логистическая регрессия без кросс-валидации,0.76
Логистическая регрессия с кросс-валидацией,0.75
Дерево решений,0.42
Случайный лес,0.72


## Вывод <a id='conclusion'></a>

In [15]:
display(result)

,f1
Логистическая регрессия без кросс-валидации,0.76
Логистическая регрессия с кросс-валидацией,0.75
Дерево решений,0.42
Случайный лес,0.72


Проведена прдобработка данных и обучена модель линейной регрессией, случайным лесом, деревом решиний.
Наилучшие показатели метрики показала модель, обученная линейной регрессией: F1 = 0.759.

Исходя из  результатов теста предлагаю модель линейной регрессией, так как удовлетворяет условию поставленной задаче.


## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны